# Classificazione binaria "selfMade" con ANN (MLPClassifier)


In [19]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import os

# Caricamento dati
X_train = pd.read_csv("../data/splitted/X_train.csv")
X_test = pd.read_csv("../data/splitted/X_test.csv")
y_train = pd.read_csv("../data/splitted/y_train.csv").values.ravel()
y_test = pd.read_csv("../data/splitted/y_test.csv").values.ravel()

In [20]:
# Pulizia dei dati
X_train = X_train.drop(columns=['source'], errors='ignore')
X_train = X_train.loc[:, X_train.nunique() > 1]
X_test = X_test[X_train.columns]

In [ ]:
# Imputazione
#sostituisco i valori nan con la media
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="mean")
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [22]:
# Standardizzazione
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

In [23]:


print("Totale NaN in X_train_scaled:", np.isnan(X_train_scaled).sum())
print("Totale NaN in X_test_scaled:", np.isnan(X_test_scaled).sum())


Totale NaN in X_train_scaled: 0
Totale NaN in X_test_scaled: 0


In [24]:
# Modello MLP
model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.001,
                      early_stopping=True, max_iter=300, random_state=42)
model.fit(X_train_scaled, y_train)


MLPClassifier(alpha=0.001, early_stopping=True, max_iter=300, random_state=42)

In [25]:
# Predizioni
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

# Metriche
train_metrics = {
    "Accuracy": accuracy_score(y_train, y_pred_train),
    "Precision": precision_score(y_train, y_pred_train),
    "Recall": recall_score(y_train, y_pred_train),
    "F1-Score": f1_score(y_train, y_pred_train),
    "Confusion Matrix": confusion_matrix(y_train, y_pred_train).tolist()
}
test_metrics = {
    "Accuracy": accuracy_score(y_test, y_pred_test),
    "Precision": precision_score(y_test, y_pred_test),
    "Recall": recall_score(y_test, y_pred_test),
    "F1-Score": f1_score(y_test, y_pred_test),
    "Confusion Matrix": confusion_matrix(y_test, y_pred_test).tolist()
}


In [26]:
# Salvataggio risultati
output_dir = "../results/classification_selfMade/ann"
os.makedirs(output_dir, exist_ok=True)

filename = os.path.join(output_dir, "ann_metrics.txt")
with open(filename, "w") as f:
    f.write("Modello: MLPClassifier (1 hidden layer da 100 neuroni, early stopping)\n\n")

    f.write("TRAIN METRICS:\n")
    for k, v in train_metrics.items():
        if k == "Confusion Matrix":
            f.write("Confusion Matrix:\n")
            f.write(f"{v}\n")
        else:
            f.write(f"{k}: {v:.4f}\n")

    f.write("\nTEST METRICS:\n")
    for k, v in test_metrics.items():
        if k == "Confusion Matrix":
            f.write("Confusion Matrix:\n")
            f.write(f"{v}\n")
        else:
            f.write(f"{k}: {v:.4f}\n")

print(f"✔ Metriche salvate in: {filename}")


✔ Metriche salvate in: ../results/classification_selfMade/ann\ann_metrics.txt


In [27]:
import matplotlib.pyplot as plt



plt.plot(model.loss_curve_)
plt.title("Loss durante il training (Un hidden layer da 100)")
plt.xlabel("Epoche")
plt.ylabel("Loss")
plt.grid(True)
plt.savefig("../results/classification_selfMade/ann/loss_curve.png")
plt.close()

In [28]:
mean_train_loss = np.mean(model.loss_curve_)
std_train_loss = np.std(model.loss_curve_)
print(f"Media loss su training set: {mean_train_loss:.4f}")
print(f"Deviazione standard loss su training set: {std_train_loss:.4f}")

Media loss su training set: 0.4671
Deviazione standard loss su training set: 0.0754


In [29]:
from sklearn.metrics import log_loss


y_proba_test = model.predict_proba(X_test_scaled)

test_loss = log_loss(y_test, y_proba_test)

print(f"Loss sul test set: {test_loss:.4f}")


Loss sul test set: 0.5582


In [30]:
from sklearn.metrics import log_loss
y_proba_train = model.predict_proba(X_train_scaled)
print("Log-loss media su training set:", log_loss(y_train, y_proba_train))


Log-loss media su training set: 0.5362032895527937


In [31]:
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
import numpy as np

# Probabilità predette sul test set
y_proba_test = model.predict_proba(X_test_scaled)

# Converto True/False in 1/0
y_test_int = y_test.astype(int)

# Calcolo della log-loss per ogni istanza
instance_losses = []
for p, true_label in zip(y_proba_test, y_test_int):
    prob = p[true_label]
    loss = -np.log(prob + 1e-15)  # epsilon per evitare log(0)
    instance_losses.append(loss)



# Grafico della loss
plt.figure(figsize=(6, 4))
plt.plot(instance_losses)
plt.title("Loss su test set (per istanza)")
plt.xlabel("Indice istanza")
plt.ylabel("Log-loss")
plt.grid(True)
plt.savefig("../results/classification_selfMade/ann/loss_curve_test.png")
plt.close()



In [32]:
print("Media log-loss sul test:", np.mean(instance_losses))
print("Deviazione standard sul test:", np.std(instance_losses))

Media log-loss sul test: 0.5581702199834775
Deviazione standard sul test: 0.48147312512594237


In [33]:
# Validazione incrociata CV
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_log_loss')
print("CV log-loss media:", -np.mean(scores))


CV log-loss media: 0.5349733988597295


In [34]:
# Ricerca per migliorare iperparametri
from sklearn.model_selection import GridSearchCV

param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50), (64, 32)],
    'alpha': [1e-5, 1e-4, 1e-3],
    'activation': ['relu', 'tanh']
}

grid = GridSearchCV(MLPClassifier(early_stopping=True, max_iter=300, random_state=42),
                    param_grid, scoring='neg_log_loss', cv=5)
grid.fit(X_train_scaled, y_train)

print("Migliori parametri:", grid.best_params_)


Migliori parametri: {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (100, 50)}
